In [1]:
import altair as alt
import polars as pl
from datetime import datetime

alt.data_transformers.enable("vegafusion")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

Exploratory analysis into across solver actions. Some initial questions are:
* Where is across directing the majority of flow to? 
* Which chains have the highest connectivity?
* Which tokens are being transported the most?

There is some initial results on token volume and by origin chain [here](https://dune.com/risk_labs/across-protocol-stats) and [here](https://dune.com/sandman2797/across-bridge-stats)

In [2]:
network_dict = {
    "name": [
        "Arbitrum", "Arbitrum Nova", "Aurora", "Avalanche", "Base", 
        "Berachain Bartio", "Blast", "Boba", "Bsc", "C1 Milkomeda", 
        "Celo", "Chiliz", "Crab", "Cyber", "Darwinia", "Ethereum Mainnet", 
        "Fantom", "Flare", "Fuji", "Galadrial Devnet", "Gnosis", 
        "Lukso", "Manta", "Mantle", "Merlin", "Metis", "Mev Commit", 
        "Mode", "Moonbeam", "Neon Evm", "Optimism", "Polygon", 
        "Polygon zkEVM", "Rsk", "Saakuru", "Scroll", "Shimmer Evm", 
        "Taiko", "Taiko Jolnr", "X Layer", "Zeta", "Zircuit", 
        "ZKsync", "Zora"
    ],
    "id": [
        42161, 42170, 1313161554, 43114, 8453, 80084, 81457, 288, 56, 2001, 
        42220, 8888, 44, 7560, 46, 1, 250, 14, 43113, 696969, 100, 
        42, 169, 5000, 4200, 1088, 17864, 34443, 1284, 245022934, 10, 137, 
        1101, 30, 7225878, 534352, 148, 167000, 1088, 196, 7000, 
        48900, 324, 7777777
    ]
}



In [3]:
network_names_df = pl.from_dict(network_dict).with_columns(pl.col('id').cast(pl.UInt64))
across_df = (
    pl.read_parquet('data/across/V3FundsDeposited/*.parquet')
    .join(network_names_df, left_on='destinationChainId', right_on='id')
    .join(network_names_df, left_on='chain_id', right_on='id', suffix='_origin')
    .rename({
        'name': 'name_destination'
    })
    .with_columns(
        pl.from_epoch('timestamp').alias('datetime')
    )
    ).filter(pl.col('datetime') > datetime(2024, 3, 31)) # 6 month dataset from April to September

In [4]:
across_df.shape

(698707, 32)

In [5]:
across_df.select('datetime').min()

datetime
datetime[μs]
2024-03-31 00:01:11


In [6]:
across_df.select('datetime').max()

datetime
datetime[μs]
2024-09-30 16:46:40


### Show Line Chart Time Series transfers to chains over time

In [7]:
time_series_to_transfers = (
    across_df.with_columns(
    pl.col('datetime').dt.round('1d').alias('date')
).group_by('name_destination', 'date').agg(
    pl.len().alias('count')
)
).filter(pl.col('name_destination').is_in(['Ethereum Mainnet', 'Optimism', 'Base', 'Arbitrum', 'Scroll', 'Blast']))

time_series_from_transfers = (
    across_df.with_columns(
    pl.col('datetime').dt.round('1d').alias('date')
).group_by('name_origin', 'date').agg(
    pl.len().alias('count')
)
)

In [8]:
time_series_to_transfers.head(5)

name_destination,date,count
str,datetime[μs],u32
"""Base""",2024-08-12 00:00:00,1881
"""Optimism""",2024-09-12 00:00:00,734
"""Optimism""",2024-08-15 00:00:00,619
"""Base""",2024-09-08 00:00:00,1669
"""Base""",2024-04-15 00:00:00,1030


In [9]:
# Rename the columns for clarity
to_transfers = time_series_to_transfers.rename({
    'count': 'to_count',
    'name_destination': 'chain'
})

from_transfers = time_series_from_transfers.rename({
    'count': 'from_count',
    'name_origin': 'chain'
})

# Combine the data on 'date' and 'chain'
combined_df = to_transfers.join(from_transfers, on=['date', 'chain'], how='full', suffix='_origin')

# Fill missing values with 0 in case there are no transfers to/from on certain dates
combined_df = combined_df.with_columns([
    pl.col('to_count').fill_null(0),
    pl.col('from_count').fill_null(0)
])

# Calculate net count (to_count - from_count)
combined_df = (combined_df
    .with_columns(
        pl.col('to_count').cast(pl.Int64),
        pl.col('from_count').cast(pl.Int64)
    )
    .with_columns(
        # counts the net transfers to the destination chain (chain)
        (pl.col('to_count') - pl.col('from_count')).alias('net_count')
)
    # replace null values with ethereum mainnet to catch transfers going out
    .with_columns(
        pl.when((pl.col('chain_origin') == "Ethereum Mainnet") & (pl.col('chain').is_null()))
        .then(pl.lit("Ethereum Mainnet"))
        .otherwise(pl.col('chain')).alias('chain')
    )
    # replace null values with ethereum mainnet to catch transfers going out
    .with_columns(
        pl.when(pl.col('date').is_null())
        .then(pl.col("date_origin"))
        .otherwise(pl.col('date')).alias('date')
    )
)

In [31]:
cumulative_net_transfer_count = combined_df.select('date', 'chain', 'net_count').sort(by='date', descending=False).with_columns(
    (pl.cum_sum('net_count').over('chain')).alias('cumulative_net_count')
)

In [43]:
alt.Chart(cumulative_net_transfer_count).mark_bar().encode(
    x=alt.X('date:T', title='Date', axis=alt.Axis(labelAngle=-45)),  # Rotate x-axis labels
    y=alt.Y('cumulative_net_count:Q', title='Cumulative Net Transfer Count', 
            axis=alt.Axis(format='.2s')),  # Format y-axis numbers as 150k, 200k, etc.
    color=alt.Color('chain:N', title='Destination Chain')  # Color based on the chain
).properties(
    width=600,
    height=400,
    title='Across Net Bridge Transfers (Cumulative)'
)

alt.Chart(...)

eth transaction hash - https://etherscan.io/tx/0xa5bab7b9975eaf2e9bb9f7d2c90a9be78b7b8731858d69ab89503737b9154adb
op transaction hash - https://optimistic.etherscan.io/tx/0xb0388a5f081f5f02b2602654cc60b7891e4a65a76fffe5152b785c82f2ac0516

- the deposit transaction is traced with a depositV3 event. The withdraw transaction from the relay is with a fillRelay event, but I am having issues getting the right event signature...

In [15]:
chain_transfer_metrics = (
    across_df
    .group_by('name_destination', 'name_origin')
    .agg(pl.len().alias('count'))
    .sort(by='count', descending=True)
)

In [41]:
alt.Chart(
    chain_transfer_metrics
    # .filter(pl.col('count') > 10000)
    # .filter(pl.col('name_destination').is_in(['Ethereum Mainnet', 'Optimism', 'Base', 'Arbitrum', 'Scroll', 'Blast']))
    ).mark_rect().encode(
    x=alt.X('name_origin:O', title='From Chain', sort=None),
    y=alt.Y('name_destination:O', title='To Chain', sort=None),
    color=alt.Color('count:Q', title='Count', scale=alt.Scale(scheme='yelloworangebrown')),
).properties(
    title='Cross-Chain Transfer Activity by Across',
    width=400,
    height=300
)

# - Ethereum -> Base
# - Ethereum -> Arbitrum
# - Optimism -> Base
# - Base -> Arbitrum

alt.Chart(...)

In [17]:
chain_transfer_metrics.tail(10)

name_destination,name_origin,count
str,str,u32
"""Blast""","""Arbitrum""",476
"""Blast""","""Scroll""",443
"""Mode""","""Arbitrum""",426
"""Polygon""","""Blast""",423
"""Mode""","""Scroll""",336
"""ZKsync""","""Blast""",324
"""Mode""","""Blast""",245
"""Zora""","""Ethereum Mainnet""",101
"""Zora""","""Blast""",30


Cross Chain Transfers by Depositor

In [18]:
cross_chain_transfer_user_profile = (
    across_df
    .group_by('name_destination', 'name_origin', 'recipient')
    .agg(pl.len().alias('count'))
    .sort(by='count', descending=True)
)

In [19]:
cross_chain_transfer_user_profile.head(10) # tried to label some random addresses. Difficult though because of lack of labeling on the explorers...
# 0x5965851f21dae82ea7c62f87fb7c57172e9f2add = Owlto_Finance
# 0xfc99f58a8974a4bc36e60e2d490bb8d72899ee9f = OKK Dex Aggregator
# 0x4fab399fcc0ecc7015bca45e0692c24e1992ff5f = LiFi Diamond (https://li.fi/) (funded by stargate)
# 0x5953c0697067d77ad48b71c5441632562c585832 = LiFi Diamond (https://li.fi/) (funded by stargate)

name_destination,name_origin,recipient,count
str,str,str,u32
"""Base""","""Ethereum Mainnet""","""0x5965851f21dae82ea7c62f87fb7c57172e9f2add""",6747
"""Polygon""","""Ethereum Mainnet""","""0x89f423567c2648bb828c3997f60c47b54f57fa6e""",2935
"""Arbitrum""","""Ethereum Mainnet""","""0xfc99f58a8974a4bc36e60e2d490bb8d72899ee9f""",2065
"""Base""","""Optimism""","""0x4fab399fcc0ecc7015bca45e0692c24e1992ff5f""",631
"""Base""","""Optimism""","""0x5953c0697067d77ad48b71c5441632562c585832""",630
"""Arbitrum""","""Optimism""","""0xed605f623fd5ee7c5ba027588fdc1a47d8d43d1a""",619
"""Base""","""Optimism""","""0x7554661ba502cc442200fa482129132fe26846ba""",612
"""ZKsync""","""Base""","""0x84e5b9260416cde351c7bc700a5fd93ca5d11a33""",598
"""ZKsync""","""Base""","""0xa6faa8fb0bf807118d29a73c7b19b18ac9cb46bd""",596


In [20]:
cross_chain_transfer_user_profile.head(10)

name_destination,name_origin,recipient,count
str,str,str,u32
"""Base""","""Ethereum Mainnet""","""0x5965851f21dae82ea7c62f87fb7c57172e9f2add""",6747
"""Polygon""","""Ethereum Mainnet""","""0x89f423567c2648bb828c3997f60c47b54f57fa6e""",2935
"""Arbitrum""","""Ethereum Mainnet""","""0xfc99f58a8974a4bc36e60e2d490bb8d72899ee9f""",2065
"""Base""","""Optimism""","""0x4fab399fcc0ecc7015bca45e0692c24e1992ff5f""",631
"""Base""","""Optimism""","""0x5953c0697067d77ad48b71c5441632562c585832""",630
"""Arbitrum""","""Optimism""","""0xed605f623fd5ee7c5ba027588fdc1a47d8d43d1a""",619
"""Base""","""Optimism""","""0x7554661ba502cc442200fa482129132fe26846ba""",612
"""ZKsync""","""Base""","""0x84e5b9260416cde351c7bc700a5fd93ca5d11a33""",598
"""ZKsync""","""Base""","""0xa6faa8fb0bf807118d29a73c7b19b18ac9cb46bd""",596


In [21]:
# this many addresses have interacted with across for bridging
cross_chain_transfer_user_profile.select('recipient').unique().shape[0]

292824

In [22]:
# this many bridge transactions have occured.
cross_chain_transfer_user_profile.select('count').sum().item()

698707

In [23]:
cross_chain_transfer_user_profile.with_columns(
    pl.when(pl.col('count') > 10).then(10).otherwise(pl.col('count')).alias('count_bins')
)

name_destination,name_origin,recipient,count,count_bins
str,str,str,u32,u32
"""Base""","""Ethereum Mainnet""","""0x5965851f21dae82ea7c62f87fb7c57172e9f2add""",6747,10
"""Polygon""","""Ethereum Mainnet""","""0x89f423567c2648bb828c3997f60c47b54f57fa6e""",2935,10
"""Arbitrum""","""Ethereum Mainnet""","""0xfc99f58a8974a4bc36e60e2d490bb8d72899ee9f""",2065,10
"""Base""","""Optimism""","""0x4fab399fcc0ecc7015bca45e0692c24e1992ff5f""",631,10
"""Base""","""Optimism""","""0x5953c0697067d77ad48b71c5441632562c585832""",630,10
…,…,…,…,…
"""ZKsync""","""Base""","""0x863054045706e3d5f95cd474ff485cbfae99db19""",1,1
"""Polygon""","""Ethereum Mainnet""","""0xe6271175509c4187887752dcfc161469422465a6""",1,1
"""Base""","""Arbitrum""","""0x2df55e0fd3a0b1f00c6de17b12d96e7351bc0744""",1,1


In [24]:
alt.Chart(cross_chain_transfer_user_profile.with_columns(
    pl.when(pl.col('count') > 8).then(8).otherwise(pl.col('count')).alias('count_bins')
)).mark_bar().encode(
    alt.X("count_bins:Q", bin=True, title='Count Bins'),
    alt.Y('count()', title='Number of Transfers'),
    alt.Color('name_destination:N', title='Destination Chain'),
).properties(
    title='Transfers to Destination Chain',
    width=600,
    height=400
)

alt.Chart(...)

In [25]:
alt.Chart(cross_chain_transfer_user_profile.with_columns(
    pl.when(pl.col('count') > 8).then(8).otherwise(pl.col('count')).alias('count_bins')
)).mark_bar().encode(
    alt.X("count_bins:Q", bin=True, title='Count Bins'),
    alt.Y('count()', title='Number of Transfers'),
    alt.Color('name_origin:N', title='Origin Chain'),
).properties(
    title='Transfers Out From Origin Chain',
    width=600,
    height=400
)

alt.Chart(...)